In [2]:
import os
os.chdir("/home/zhuw10/CL/SV/prj_bench/notebooks" )

# this is a test


1. Import the fai as the interval tree to be included.
I may need to refer this part to the genome_tree.py

In [54]:
import pandas as pd
fai_fn = "../../refGenomes/Homo_sapiens_assembly19.fasta.fai"
chrs = list(map(str, range(1,23)))+['X','Y','MT']

fai_file=open(fai_fn)
fai_dat = pd.read_csv(fai_fn, sep='\t',header=None)
fai_dat[fai_dat[0].isin(chrs)]





,0,1,2,3,4
0,1,249250621,52,80,81
1,2,243199373,252366358,80,81
2,3,198022430,498605776,80,81
3,4,191154276,699103539,80,81
4,5,180915260,892647296,80,81
5,6,171115067,1075824049,80,81
6,7,159138663,1249078107,80,81
7,8,146364022,1410206056,80,81
8,9,141213431,1558399681,80,81
9,10,135534747,1701378334,80,81


In [55]:
from intervaltree import IntervalTree, Interval
from collections import defaultdict, namedtuple

included_regs = defaultdict(IntervalTree)
fai_dat[fai_dat[0].isin(chrs)].apply( lambda x: included_regs[x[0]].addi(0,x[1]), axis=1)

included_regs


defaultdict(intervaltree.intervaltree.IntervalTree,
            {'1': IntervalTree([Interval(0, 249250621)]),
             '2': IntervalTree([Interval(0, 243199373)]),
             '3': IntervalTree([Interval(0, 198022430)]),
             '4': IntervalTree([Interval(0, 191154276)]),
             '5': IntervalTree([Interval(0, 180915260)]),
             '6': IntervalTree([Interval(0, 171115067)]),
             '7': IntervalTree([Interval(0, 159138663)]),
             '8': IntervalTree([Interval(0, 146364022)]),
             '9': IntervalTree([Interval(0, 141213431)]),
             '10': IntervalTree([Interval(0, 135534747)]),
             '11': IntervalTree([Interval(0, 135006516)]),
             '12': IntervalTree([Interval(0, 133851895)]),
             '13': IntervalTree([Interval(0, 115169878)]),
             '14': IntervalTree([Interval(0, 107349540)]),
             '15': IntervalTree([Interval(0, 102531392)]),
             '16': IntervalTree([Interval(0, 90354753)]),
             '

## 2. Load the truth vcf 
build another interval tree
+ all interval should be within the included regions
+ drop it if it is not intra SV
+ drop it if it is too small or too large
    + >=50
    + <=50000


In [56]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

import pysam
import pprint
vcf_fn = "../data/dream_modified.vcf.gz"
vcf = pysam.VariantFile(vcf_fn)
n=0;
c=0

def filter_base_entry(entry, included_regs, ALTS=['<INV>', '<DUP>', '<DEL>', '<INS>']):
    ''' 
    I may define excluded_regs instead so as to drop any entry overlaps with the excluded regions
    '''
    if entry.alts[0] not in ALTS:
        return False

    # it is a set
    regs = included_regs[entry.chrom].overlap(entry.start, entry.stop)
    if len(regs)!=1:
        print("Error in: %s" % (entry))
        return False
    if regs.pop().contains_interval(Interval(entry.start, entry.stop)):
        return True
    else:
        return False 



for entry in vcf.fetch():
    n+=1
    if filter_base_entry(entry, included_regs) :
        print(entry.alts[0], entry.start, entry.stop)
        c+=1

print(n,c)



<DUP> 25204182 25210854
<INV> 32176186 32193834
<DEL> 36419037 36421454
<INV> 37852934 37863686
<DEL> 40302119 40310286
<DEL> 52748187 52752298
<DEL> 62421145 62424426
<INV> 67068898 67075815
<DUP> 67694632 67698300
<DUP> 70709636 70717682
<DUP> 111418702 111426908
<DUP> 162165374 162168815
<DEL> 162739418 162742026
<INV> 167044909 167047890
<INV> 167450586 167455866
<DUP> 172023645 172029939
<INV> 174098254 174103055
<INV> 178678992 178680869
<DEL> 181069590 181072820
<DEL> 211813609 211816889
<INV> 212816043 212836430
<DEL> 222412147 222423755
<DUP> 232213568 232219117
<INV> 247204800 247209976
<INV> 247874052 247886918
<DUP> 5304636 5309227
<DUP> 7115151 7139773
<DEL> 12504806 12506154
<DUP> 19431941 19433784
<DUP> 23368210 23378594
<DEL> 36026185 36030048
<INV> 37564758 37573259
<DUP> 44017954 44025265
<DUP> 61282470 61291407
<DUP> 75225370 75229777
<DEL> 77154046 77175071
<DEL> 78239538 78247544
<DEL> 84044870 84048559
<INV> 87483048 87496843
<INV> 116043741 116049168
<INV> 120317

In [58]:
pp.pprint(dir(vcf))


[   '__class__',
    '__delattr__',
    '__dir__',
    '__doc__',
    '__enter__',
    '__eq__',
    '__exit__',
    '__format__',
    '__ge__',
    '__getattribute__',
    '__gt__',
    '__hash__',
    '__init__',
    '__init_subclass__',
    '__iter__',
    '__le__',
    '__lt__',
    '__ne__',
    '__new__',
    '__next__',
    '__pyx_vtable__',
    '__reduce__',
    '__reduce_ex__',
    '__repr__',
    '__setattr__',
    '__setstate__',
    '__sizeof__',
    '__str__',
    '__subclasshook__',
    'add_hts_options',
    'category',
    'check_truncation',
    'close',
    'closed',
    'compression',
    'copy',
    'description',
    'drop_samples',
    'duplicate_filehandle',
    'fetch',
    'filename',
    'format',
    'get_reference_name',
    'get_tid',
    'header',
    'header_written',
    'index',
    'index_filename',
    'is_bam',
    'is_bcf',
    'is_closed',
    'is_cram',
    'is_open',
    'is_read',
    'is_reading',
    'is_remote',
    'is_sam',
    'is_stream',

In [61]:
# can i fetch again wo reset: yes.  there i 
z=0
for entry in vcf.fetch():
    z+=1

print(z)

4280


## 3. Now we process the prediction 

In [65]:
!cp ../../test/IS1_4_new/output/compare_and_annotate/delly/bed_input/*  ../data/
!ls -al ../data

total 36369
drwxrwxr-x 2 zhuw10 root      794 Jun 12 23:07 .
drwxrwxr-x 5 zhuw10 root       70 Jun 12 09:57 ..
-rw-rw-r-- 1 zhuw10 root    52385 Jun 11 18:23 dream_modified.vcf.gz
-rw-rw-r-- 1 zhuw10 root    46117 Jun 11 18:23 dream_modified.vcf.gz.tbi
-rw-rw-r-- 1 zhuw10 root    51675 Jun 11 16:42 dreamsmc.IS1.updateSV.truth.vcf.gz
-rw-rw-r-- 1 zhuw10 root    46149 Jun 11 16:42 dreamsmc.IS1.updateSV.truth.vcf.gz.tbi
-rw-rw-r-- 1 zhuw10 root   725960 Jun 11 16:46 HG002_SVs_Tier1_v0.6.bed
-rw-rw-r-- 1 zhuw10 root 25738211 Jun 11 16:46 HG002_SVs_Tier1_v0.6.vcf.gz
-rw-rw-r-- 1 zhuw10 root   451795 Jun 11 16:49 HG002_SVs_Tier1_v0.6.vcf.gz.tbi
-rw-rw-r-- 1 zhuw10 root      860 Jun 12 23:07 IS1_end1.bed
-rw-rw-r-- 1 zhuw10 root      872 Jun 12 23:07 IS1_end2.bed
-rw-rw-r-- 1 zhuw10 root    21300 Jun 12 23:07 IS1_intra.bed
-rw-rw-r-- 1 zhuw10 root   316935 Jun 12 23:07 IS1_orig
-rw-rw-r-- 1 zhuw10 root     5487 Jun 12 23:07 IS2_end1.bed
-rw-rw-r-- 1 zhuw10 root     5504 Jun 12 23:07 IS2_end2.

In [78]:
# we start with IS1_orig
bed_input = pd.read_csv("../data/IS1_intra.bed", header=None, sep="\t", names=['chrom', "start", "end", "id"])
bed_input

# filter out if the interval is out of the included region or too short, too long, etc
def filter_comp_bed(bed_row, included_regs):
    chrom, start, stop, id = bed_row
    regs = included_regs[chrom].overlap(start, stop)
    if len(regs)==0:
        return False # like those GL
    if len(regs)>1:
        print("Error in: %s" % (bed_row))
        return False
    if regs.pop().contains_interval(Interval(start, stop)):
        return True
    else:
        return False 



is_selected = bed_input.apply(lambda x: filter_comp_bed(x, included_regs), axis=1)

is_selected.value_counts()

bed_fil = bed_input[is_selected]

# make the interval tree for comp bed file
comp_svs = defaultdict(IntervalTree)
bed_fil.apply( lambda x: comp_svs[x[0]].addi(x[1],x[2],x[3]), axis=1)
comp_svs

7, 73393251, 319), Interval(76310321, 76310621, 320), Interval(80607174, 80607509, 321)]),
             '18': IntervalTree([Interval(6289571, 6294746, 322), Interval(6289616, 6294770, 323), Interval(11981835, 11982135, 324), Interval(25269299, 25277175, 326), Interval(25269352, 25277201, 327), Interval(25981136, 25988364, 328), Interval(25981162, 25988350, 329), Interval(26686738, 26687084, 330), Interval(30594087, 30599292, 331), Interval(30594134, 30599309, 332), Interval(31654780, 31668568, 333), Interval(33026647, 33028563, 334), Interval(33738313, 33738659, 335), Interval(36514778, 36515098, 336), Interval(43529320, 43532210, 337), Interval(44702538, 44702875, 338), Interval(49938727, 49939077, 339), Interval(57292799, 57293101, 340), Interval(64796797, 64797106, 341), Interval(66168156, 66168471, 342), Interval(72863311, 72871308, 343), Interval(72863321, 72871361, 344), Interval(73870872, 73871218, 345), Interval(74334794, 74335104, 346), Interval(74835155, 74839897, 347), Inter

In [76]:
is_selected.value_counts()

True     857
False      3
dtype: int64

In [98]:
# Now start to check eash entry in base vcf to make sure whether there is a match or not.
def fetch_coords(lookup, entry, dist=500):
    ''' 
     
    Get the minimum/maximum fetch coordinates to find all variants within dist of variant
    
    similar to fetch_coords defined in truvari
    see https://github.com/spiralgenetics/truvari/blob/e84804b20214775ba625819568e74a1920c99a06/truvari/comparisons.py#L159
    '''
    start, end = entry.start, entry.stop
    start -= dist
    end += dist
    # Membership queries are fastest O(1)
    if not lookup[entry.chrom].overlaps(start, end):
        return None, None

    cand_intervals = lookup[entry.chrom].overlap(start, end)
    s_ret = min([x.begin for x in cand_intervals if x.overlaps(start, end)])
    e_ret = max([x.end for x in cand_intervals if x.overlaps(start, end)])
    return s_ret, e_ret

MATCHRES = namedtuple("matchresult", ("base", "comp", "ovl_pct"))

def reciprocal_overlap(astart, aend, bstart, bend):
    """
    creates a reciprocal overlap rule for matching two entries. Returns a method that can be used as a match operator
    """
    ovl_start = max(astart, bstart)
    ovl_end = min(aend, bend)
    if ovl_start < ovl_end:  # Otherwise, they're not overlapping
        ovl_pct = float(ovl_end - ovl_start) / max(aend - astart, bend - bstart)
    else:
        ovl_pct = 0
    return ovl_pct

def get_match(entry, interval, pad=100):
    '''
    compare the base entry and the comp call
    return None if there is no match 
    and the match information, metrics otherwise

    similar to match_calls() in truvari
    '''
    if not interval.overlaps(entry.start-pad, entry.stop+pad):
        return None

    #print(dir(interval))
    # ovs = interval.overlap_size(entry.start, entry.stop)
    ovl_pct = reciprocal_overlap(entry.start, entry.stop, interval.begin, interval.end)
    return MATCHRES(entry, interval, ovl_pct)

fn=0
tp=0
for entry in vcf.fetch():
    if not filter_base_entry(entry, included_regs) :
        continue

    
    fetch_start, fetch_end = fetch_coords(comp_svs, entry)
    if fetch_start is None and fetch_end is None:
        # print("FN: ", entry)
        fn+=1
        continue

    mats = []
    # now get the matched calls from the given region
    for comp_sv in comp_svs[entry.chrom].overlap(fetch_start, fetch_end):
        mat = get_match(entry, comp_sv)
        if mat is None:
            continue

        # print(mat.base.chrom, mat.base.start, mat.base.stop, mat.comp.begin, mat.comp.end, mat.ovl_pct)
        mats.append(mat)

    # select the best match
    if len(mats)>0:
        mats.sort(reverse=True, key=lambda x:x.ovl_pct)
        mat = mats[0]
        print(mat.base.chrom, mat.base.start, mat.base.stop, mat.comp.begin, mat.comp.end, mat.ovl_pct)
        tp+=1

print(fn,tp)


1 25204182 25210854 25204175 25210819 0.9947541966426858
1 32176186 32193834 32176192 32193830 0.9994333635539437
1 36419037 36421454 36418919 36421450 0.9533781114184117
1 37852934 37863686 37852932 37863715 0.9971251043308912
1 40302119 40310286 40302027 40310309 0.9861144651050471
1 52748187 52752298 52748054 52752290 0.9686024551463644
1 62421145 62424426 27357068 79152194 6.334572870814138e-05
1 67068898 67075815 67068918 67075807 0.9959520023131415
1 67694632 67698300 67694620 67698183 0.968102508178844
1 70709636 70717682 27357068 79152194 0.00015534280194626806
1 111418702 111426908 111418762 111426825 0.982573726541555
1 162165374 162168815 162165380 162168792 0.991572217378669
1 167044909 167047890 167044910 167047887 0.9986581683998658
1 172023645 172029939 168186264 182274311 0.00044676171225152783
1 174098254 174103055 174098278 174103122 0.986168455821635
1 178678992 178680869 168186264 182274311 0.00013323351348842037
1 181069590 181072820 181069580 181072842 0.990190067

NameError: name 'cand_intervals' is not defined

# Now process the GIAB vcf file


In [19]:
import pysam
import os
import sys

os.chdir("/home/zhuw10/CL/SV/prj_bench/notebooks" )
giab_fn = "../data/HG002_SVs_Tier1_v0.6.vcf.gz"
giab_vcf = pysam.VariantFile(giab_fn)

In [24]:
for entry in giab_vcf.fetch():
    if "SVTYPE" in entry.info:
        print(entry.info["SVTYPE"])
    sys.exit()

DEL


SystemExit: 